# Summarise building density information

Access NISMOD-DB to download building data and summarise regional floor area and footprint density.

In [ ]:
import configparser
import glob
import json
import os

import pandas
import geopandas
import requests
import shapely.wkt

In [ ]:
def get_auth():    
    # Read connection details
    if 'NISMOD_API_USER' in os.environ and 'NISMOD_API_PASSWORD' in os.environ:
        username = os.environ['NISMOD_API_USER']
        password = os.environ['NISMOD_API_PASSWORD']
    else:
        parser = configparser.ConfigParser()
        parser.read('dbconfig.ini')
        username = parser['nismod-api']['user']
        password = parser['nismod-api']['password']

    return (username, password)

In [ ]:
LAD_CODE = 'E06000042'
BUILDINGS_YEAR = 2017
CACHE_PATH = os.path.join('.', 'db-data')
AUTH = get_auth()

In [ ]:
def get_buildings(auth, lad_code, year):
    try:
        os.mkdir(os.path.join(CACHE_PATH, lad_code))
    except FileExistsError:
        pass
    buildings_file = os.path.join(CACHE_PATH, lad_code, "buildings_{}.json".format(lad_code))

    if not os.path.exists(buildings_file):
        r = requests.get(
            'https://www.nismod.ac.uk/api/data/mastermap/buildings',
            auth=auth,
            params={
                'scale': 'lad',
                'area_codes': lad_code,
                'building_year': year
            },
            stream=True
        )
        with open(buildings_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

In [ ]:
get_buildings(AUTH, LAD_CODE, BUILDINGS_YEAR)

In [ ]:
with open(os.path.join(CACHE_PATH, LAD_CODE, "buildings_{}.json".format(LAD_CODE))) as fh:
    lad_buildings = json.load(fh)
lad_buildings

In [ ]:
df = geopandas.GeoDataFrame(lad_buildings)
df.geometry = df.geom.apply(lambda wkt: shapely.wkt.loads(wkt))
df

In [ ]:
df.columns

In [ ]:
len(df)

In [ ]:
df.oa.unique()

In [ ]:
summary = df[
    ['oa', 'mistral_function_class', 'mistral_building_class', 'floor_area', 'footprint_area', 'res_count']
].groupby(
    ['oa', 'mistral_function_class', 'mistral_building_class']
).sum()
summary

In [ ]:
summary.to_csv('summary_E06000042.csv')

In [ ]:
df.drop('geom', axis=1).to_csv('buildings_E06000042.csv', index=False)